# 分子動力学プログラム LAMMPS を Amazon SageMaker Processing で GPU を用いて動かすサンプル

* [LAMMPS](https://www.lammps.org/) を Amazon SageMaker Processing で動かす。細かい情報は下記を参照。
    * [LAMMPSのマニュアル](https://docs.lammps.org/Manual.html)
    * [ポリエチレン分子をシミュレーション](https://winmostar.com/jp/tutorials/LAMMPS_tutorial_8%28Polymer_Elongation%29.pdf)
    * [SageMaker SDK doc](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_processing.html)
    * [SageMaker Processing 開発者ガイド](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)

このノートブックは Amazon SageMaker Notebook(≠Studio) の GPU インスタンス(g4dn.xlargeなど)で実行することを前提とする。理由は以下の通り。

1. docker コマンドを使うため(Studioを使う場合は sm-docker コマンドに修正の必要あり)
2. ビルドしたコンテナをローカル(SageMaker Notebook内)でテスト実行するのにあたり、GPU で動かす必要があるため

In [ ]:
pip install sagemaker -U

In [ ]:
# 使用するライブラリを読み込み
import boto3, sagemaker, os, shutil, re
from sagemaker.tensorflow import TensorFlow
import pandas as pd
from matplotlib import pyplot as plt
from io import StringIO

## コンテナイメージのビルド
まずは SageMaker Processing で LAMMPS が動かせるよう、コンテナイメージの中で LAMMPS をビルドする。  
### ビルド環境のセットアップ
ビルドは、`/var/lib/docker`を利用するが、SageMaker Notebook では該当領域は`/` にマウントされた 15GB では、ビルドに耐えられないので、別途 EBS をマウントしている`/home/ec2-user/SageMaker`以下の領域を使うように変更するスクリプトを実行する

In [ ]:
cat ./setup.sh

In [ ]:
!./setup.sh

In [ ]:
!sudo /etc/init.d/docker stop
!sudo mv /var/lib/docker /home/ec2-user/SageMaker/docker
!sudo ln -s /home/ec2-user/SageMaker/docker /var/lib/docker
!sudo /etc/init.d/docker start

### ビルド

In [ ]:
%%time

IMAGE_NAME = 'lampps'
TAG=':v1'

!$(aws ecr get-login --region ap-northeast-1 --registry-ids 763104351884 --no-include-email)

%cd ./container
# !docker stop $(docker ps -q)
# !docker rm $(docker ps -q -a)
# !docker rmi -f $(docker images -a -q)
!docker build -t {IMAGE_NAME}{TAG} .
%cd ../

### Image Test
ビルドしたイメージをテストする。  
このインスタンスでコンテナを動かし、mpirunコマンド(`./lmp_equiliv.sh`に内包)を実行し実際にシミュレーションを行う。  

jupyter notebook の new -> terminal から下記セルの出力をコピーして実行

In [ ]:
command_str = f"""
cd {os.getcwd()} # clone した先のディレクトリに cd

docker run --gpus all -v {os.getcwd()}/test/:/test -it --rm --entrypoint "bash" {IMAGE_NAME}{TAG}

cd /test

./lmp_equiliv.sh

tail -n1 lmp_equiliv.log # 実行時間が出ていたらOK, 概ね g4dn.xlarge で 6 分弱で完了

tail -n1 lmp2data_equiliv.log  # finish replica が出ていたらOK

exit # 終わったら exit でコンテナから抜ける(コンテナも終了する)
"""
print(command_str)

### Push

In [ ]:
%%time

MY_ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
# PUBLIC_ACCOUNT_ID = '763104351884'

REGION = boto3.session.Session().region_name

MY_ECR_ENDPOINT = f'{MY_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'



MY_REPOSITORY_URI = f'{MY_ECR_ENDPOINT}sagemaker-{IMAGE_NAME}'
MY_IMAGE_URI = f'{MY_REPOSITORY_URI}{TAG}'

!$(aws ecr get-login --region {REGION} --registry-ids {MY_ACCOUNT_ID} --no-include-email)
 
# リポジトリの作成
# !aws ecr delete-repository --repository-name {IMAGE_NAME} --force # 同名のリポジトリがあった場合削除
# !aws ecr create-repository --repository-name sagemaker-{IMAGE_NAME}
 
# !docker tag {IMAGE_NAME}{TAG} {MY_IMAGE_URI}
# !docker push {MY_IMAGE_URI}

print(f'コンテナイメージは {MY_IMAGE_URI} へ登録されています。')

## SageMaker Processing で ポリエチレンのシミュレーション
* コンテナイメージの作成が完了したので、シミュレーションのジョブを投入する
* 以降重い処理はこのノートブックインスタンスで実行することはなく、ただジョブの投入を行うだけなので、t3.mediumなどでも十分動く

### 必要なデータ(パラメータファイルなど)を S3 にアップロード

In [ ]:
prefix = 'lammps_simple_training'
input_s3_uri = sagemaker.session.Session().upload_data(path='param/', key_prefix=prefix)
print(input_s3_uri)

### ローカルモードでシミュレーションを実行
* debug 用途
    * トレーニングインスタンスは立ち上げに時間がかかるため、ノートブックインスタンスでトレーニングジョブを動かすことができる
    * 本来ノートブックインスタンスは低スペック（コード記述とAPI実行用）なため、ローカルモードでの本番運用はすべきでない
* src/run.py の実行確認
* シミュレーションは軽めにしておいて、run.pyの動作確認を行う

In [ ]:
local_estimator = sagemaker.tensorflow.TensorFlow(
    base_job_name=BASE_JOB_NAME,
    entry_point='src/run.py',
    role=sagemaker.get_execution_role(),
    image_uri=MY_IMAGE_URI,
    instance_count=1,
    instance_type='local',
    hyperparameters={
        'np':'2',
        'gpu':'1'
    }
)
local_estimator.fit({
    'param': input_s3_uri
})

### トレーニングインスタンスでシミュレーションを実行
* GPU インスタンスで実行
* 成功していたら最後に `finish replica` と表示される

In [ ]:
BASE_JOB_NAME='LAMMPS-polyethylene-simple'
estimator = TensorFlow(
    base_job_name=BASE_JOB_NAME,
    entry_point='src/run.py',
    role=sagemaker.get_execution_role(),
    image_uri=MY_IMAGE_URI,
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    hyperparameters={
        'np':'2',
        'gpu':'1'
    }
)
estimator.fit(inputs={'param': input_s3_uri})

## シミュレーション結果の確認
### ジョブの結果詳細確認

In [ ]:
# 実行したジョブの詳細確認
estimator.latest_training_job.describe()

### ジョブの出力結果を取得

In [ ]:
job_name = estimator.latest_training_job.describe()['TrainingJobName']
output_s3_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
print(job_name, output_s3_uri)

In [ ]:
# ローカル(ノートブックインスタンス)にダウンロード
!mkdir ./{job_name}
!aws s3 cp {output_s3_uri} ./{job_name}
!tar zxvf {job_name}/model.tar.gz -C ./{job_name}

## (応用編1)ポリエチレンのシミュレーションをグリッドサーチ
* パラメータを変えながら一気にジョブを投入する
* シミュレーションの開始時の温度を 2パターン (540K,550K)用意して、一気にジョブを流す
* param_base/lmp_equliv.in に変数を変えるためのプレースホルダを事前に用意し、実行する時に書き換えてジョブを流す

In [ ]:
# tmpe1_1_variable という文字列がプレースホルダで、動的に書き換える
!head -n12 ./param_base/lmp_equiliv.in

In [ ]:
prefix = 'lammps_grid_training'
input_s3_uri = sagemaker.session.Session().upload_data(path='param_base/', key_prefix=prefix)
print(input_s3_uri)
BASE_JOB_NAME='LAMMPS-polyethylene-grid'
estimators = []
for i,kervin in enumerate([540,550]):
    estimator = TensorFlow(
        base_job_name=BASE_JOB_NAME,
        entry_point='src_grid/run.py',
        role=sagemaker.get_execution_role(),
        image_uri=MY_IMAGE_URI,
        instance_count=1,
        instance_type='ml.g4dn.xlarge',
        hyperparameters={
            'np':'2',
            'gpu':'1',
            'tempe1-1-variable': str(kervin),
            'tempe1-2-variable':'550',
            'dt1-variable'     :'1.6',
            'nrun1-variable'   :'ceil(1e5)'
        }
    )
    estimator.fit(
        inputs={'param': input_s3_uri},
        wait=False
    )
    estimators.append(estimator)

In [ ]:
# ジョブが終わるまで待つ
for estimator in estimators:
    for job in estimator.jobs:
        job.wait()

In [ ]:
# ジョブの結果を手元にダウンロード
for estimator in estimators:
    for job in estimator.jobs:
        job_name = job.describe()['TrainingJobName']
        output_s3_uri = job.describe()['ModelArtifacts']['S3ModelArtifacts']
        !mkdir -p {job_name}
        !aws s3 cp {output_s3_uri} {job_name}
        !tar zxvf {job_name}/model.tar.gz -C ./{job_name}

In [ ]:
# 可視化
dfs = {}
for estimator in estimators:
    for job in estimator.jobs:
        directory = job.describe()['TrainingJobName']
        dfs[directory] = pd.read_csv(f'{directory}/1.csv')
fig = plt.figure(figsize=(20,16))
for i,column in enumerate(dfs[directory].columns[1:]):
    ax = fig.add_subplot(3,3,i+1)
    ax.set_title(column)
    for key in dfs:
        ax.plot(dfs[key]['Step'],dfs[key][column],label=key)
    ax.legend()

## SageMaker Experiments を用いてシミュレーションの実験管理を行う
* シミュレーションの入力と出力を一元管理

In [ ]:
# sagemaker experiments SDK のインストール
!pip install sagemaker-experiments

In [ ]:
from smexperiments import experiment
from smexperiments.experiment import Experiment
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
import time

In [ ]:
my_lammps_experiment = experiment.Experiment.create(experiment_name='my-LAMMPS-experiments')

In [ ]:
my_trial = my_lammps_experiment.create_trial(trial_name='simple-trial')

In [ ]:
# 標準出力から回収する定義

metric_definitions = [
    {'Name':'Step','Regex': '([0-9\\.]+)'},
    {'Name':'Time','Regex': '([0-9\\.]+)'},
    {'Name':'Temp','Regex': '([0-9\\.]+)'},
    {'Name':'PotEng','Regex': '([0-9\\.]+)'},
    {'Name':'KinEng','Regex': '([0-9\\.]+)'},
    {'Name':'TotEng','Regex': '([0-9\\.]+)'},
    {'Name':'Enthalpy','Regex': '([0-9\\.]+)'},
    {'Name':'Press','Regex': '([0-9\\.]+)'},
    {'Name':'Volume','Regex': '([0-9\\.]+)'},
    {'Name':'Density','Regex': '([0-9\\.]+)'}
]

In [ ]:
prefix = 'lammps_grid_training'
input_s3_uri = sagemaker.session.Session().upload_data(path='param_base/', key_prefix=prefix)
print(input_s3_uri)
BASE_JOB_NAME='LAMMPS-polyethylene-grid'
estimators = []
for i,kervin in enumerate([540,550]):
    estimator = TensorFlow(
        base_job_name=BASE_JOB_NAME,
        entry_point='src_experiments/run.py',
        role=sagemaker.get_execution_role(),
        image_uri=MY_IMAGE_URI,
        instance_count=1,
        instance_type='ml.g4dn.xlarge',
        hyperparameters={
            'np':'2',
            'gpu':'1',
            'tempe1-1-variable': str(kervin),
            'tempe1-2-variable':'550',
            'dt1-variable'     :'1.6',
            'nrun1-variable'   :'ceil(1e5)'
        },
        metric_definitions=metric_definitions, # metrics 定義を指定
    )
    estimator.fit(
        inputs={'param': input_s3_uri},
        wait=False,
        experiment_config={
            "ExperimentName": my_lammps_experiment.experiment_name,
            "TrialName": my_trial.trial_name,
            "TrialComponentDisplayName": "polyethylene-simulation"
        }
    )
    estimators.append(estimator)

In [ ]:
# ジョブが終わるまで待つ
for estimator in estimators:
    for job in estimator.jobs:
        job.wait()

In [ ]:
trial_component_analytics = ExperimentAnalytics(
    experiment_name=my_lammps_experiment.experiment_name,
)

analytic_table = trial_component_analytics.dataframe()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
analytic_table

In [ ]:
def cleanup_sme_sdk(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
        experiment_name = experiment.experiment_name
    experiment.delete()
    print(f"\nExperiment {experiment_name} deleted")

In [ ]:
experiment_to_cleanup = Experiment.load(
    # Use experiment name not display name
    experiment_name=my_lammps_experiment.experiment_name)

cleanup_sme_sdk(experiment_to_cleanup)

In [ ]:
sm_client = boto3.client('sagemaker')

In [ ]:
processing_job_list = sm_client.list_processing_jobs(SortBy='CreationTime',SortOrder='Descending',MaxResults=1)['ProcessingJobSummaries']

In [ ]:
for job in processing_job_list:
    if 'LAMMPS' in job['ProcessingJobName']:
        print(sm_client.describe_processing_job(ProcessingJobName=job['ProcessingJobName']))

In [ ]:
job_description = sm_client.describe_processing_job(ProcessingJobName=job['ProcessingJobName'])
job_description

In [ ]:
job_description['AppSpecification']